In [0]:
display(dbutils.fs.ls("/databricks-datasets/"))


path,name,size,modificationTime
dbfs:/databricks-datasets/COVID/,COVID/,0,1732267479968
dbfs:/databricks-datasets/README.md,README.md,976,1561418533000
dbfs:/databricks-datasets/Rdatasets/,Rdatasets/,0,1732267479968
dbfs:/databricks-datasets/SPARK_README.md,SPARK_README.md,3359,1561418605000
dbfs:/databricks-datasets/adult/,adult/,0,1732267479968
dbfs:/databricks-datasets/airlines/,airlines/,0,1732267479968
dbfs:/databricks-datasets/amazon/,amazon/,0,1732267479968
dbfs:/databricks-datasets/asa/,asa/,0,1732267479968
dbfs:/databricks-datasets/atlas_higgs/,atlas_higgs/,0,1732267479968
dbfs:/databricks-datasets/bikeSharing/,bikeSharing/,0,1732267479968


In [0]:
display(dbutils.fs.ls("/databricks-datasets/nyctaxi/sample/"))


path,name,size,modificationTime
dbfs:/databricks-datasets/nyctaxi/sample/README.md,README.md,67,1617329156000
dbfs:/databricks-datasets/nyctaxi/sample/json/,json/,0,1732267480816


# Bronze

In [0]:
raw_data = spark.read.format("json").option("inferSchema", "true").load("/databricks-datasets/nyctaxi/sample/json")
raw_data.show()


+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+-------------------+
|DOLocationID|PULocationID|RatecodeID|VendorID|congestion_surcharge|extra|fare_amount|improvement_surcharge|mta_tax|passenger_count|payment_type|store_and_fwd_flag|tip_amount|tolls_amount|total_amount|tpep_dropoff_datetime|tpep_pickup_datetime|trip_distance|pep_pickup_date_txt|
+------------+------------+----------+--------+--------------------+-----+-----------+---------------------+-------+---------------+------------+------------------+----------+------------+------------+---------------------+--------------------+-------------+-------------------+
|         236|         132|         2|       2|                 2.5|  0.0|       52.0|                  0.3|    0.5|              1|           1|                 N

# Silver

In [0]:
from pyspark.sql.functions import col

silver_data = raw_data.filter("trip_distance > 0 AND fare_amount > 0")

# add col pour la durée du trajet
silver_data = silver_data.withColumn("trip_duration", col("tpep_dropoff_datetime").cast("long") - col("tpep_pickup_datetime").cast("long"))

# Sauvegarde de la table Silver
silver_data.write.format("delta").mode("overwrite").saveAsTable("silver_taxi")


In [0]:
silver_data.printSchema()


root
 |-- DOLocationID: long (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- VendorID: long (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pep_pickup_date_txt: date (nullable = true)
 |-- trip_duration: long (nullable = true)



# Gold

In [0]:
from pyspark.sql.functions import avg, count, desc

gold_data = silver_data.groupBy("PULocationID").agg(
    avg("fare_amount").alias("avg_fare"),
    avg("trip_distance").alias("avg_distance"),
    count("*").alias("total_trips")
)

# Sauvegarde des résultats dans la table Gold
gold_data.write.format("delta").mode("overwrite").saveAsTable("gold_taxi")


# Partie 4

In [0]:
%sql
SELECT PULocationID, avg_fare, total_trips
FROM gold_taxi
ORDER BY avg_fare DESC
LIMIT 10;


PULocationID,avg_fare,total_trips
1,85.9683520599251,267
99,80.5,2
23,77.47641025641026,39
5,74.421875,32
176,73.0,1
118,68.1125,40
187,61.00090909090909,11
206,59.42250000000001,12
265,58.30913253012047,1660
156,56.0625,8
